In [1]:
!pip install python-snap7==0.5
# va en version 1.1
import snap7.client as c
from snap7.util import * #set_int   set_bool
from snap7.snap7types import * # areas
import time
# !pip install keyboard 
import keyboard  # using module keyboard
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Excitación del módulo  multivariable SIEMENS del PCJIC

En el laboratorio de automática del Politécnico Colombiano Jaime Isaza Cadavid se encuentra un módulo multivariable que está controlado con un PLC  SIEMENS. Por esta razón, se utiliza el módulo `SNAP7` para interactuar con el PLC desde el notebook.

Para instalar `SNAP7`:
- Use la instrucción 
~~~
!pip install python-snap7==0.5
~~~



- Descargue la librería completa de [https://sourceforge.net/projects/snap7/files/1.2.1/](https://sourceforge.net/projects/snap7/files/1.2.1/)

- Copie `snap7.dll` y `snap7.lib` a su carpeta `System32`.



## 1. Establezca conexión con el PLC.

El PLC tiene configurada la dirección IP **192.168.0.1**. Este funcionará como un servidor al cuál debe conectar su PC configurándole una dirección estática dentro de la subred del PLC.  Su dirección puede ser **192.168.0.2**, por ejemplo. Una vez su equipo haya establecido comunicación con el PLC, establezca conexión desde el notebook con las siguientes instrucciones.

In [2]:
#Conexión con el PLC
plc = c.Client()
IP_PLC = '192.168.0.1'

In [3]:
try:
    plc.connect(IP_PLC,0,1)
    print("Conectado")
except:
    print("Algo no funcionó")

Conectado


## 2. Identifique actuadores y sensores.

Con `SNAP7` puede manipular los actuadores del sistema y capturar los datos de los sensores. Se han construido algunas funciones para facilitar la interacción con el PLC. 

Tenga en cuenta la siguiente información, que le orientará en cómo se ha conectado el sistema al PLC.

|Elemento   |  Punto  |
|-----------|---------|
| Presión 	|    IW96 |
| Flujo		|    IW98 |
| Nivel		|   IW100 |
| Run Bomba	|    Q0.5 |
| Bomba		|    QW80 |
| Válvula H20 |  QW96 |

In [4]:
#ESCRIBIR SALIDA BOOLEANA
def escr_sal_bool(byte, bit, valor):
    lectura = plc.ab_read(byte, bit)
    set_bool(lectura, byte, bit, valor)
    plc.ab_write(0, lectura)
#escr_sal_bool(0,1,1)

#ESCRIBIR SALIDA ENTERO
def escr_sal_ent(byte,valor):
    lectura = plc.read_area(areas['PA'], 0, byte, 2) #PA: salidas, 0: bloque de datos, dirección, # bytes.
    # print(lectura)
    set_int(lectura, 0, valor)  # se da formato al valor deseado, en este caso entero
    plc.write_area(areas['PA'], 0, byte, lectura)  # Escribe en la dirección definida
#escr_sal_ent(90,9000)

#LEER MARCA ENTERA
def leer_ent_ent(byte):
    leer = plc.read_area(areas['PE'],0,byte,2) #PE: entradas, 0: bloque de datos, dirección, # bytes.
    leer_ent = get_int(leer,0) #Comando get_int(_bytearray, byte_index)
    return leer_ent

def AbrirValvulaQW96(valvulap):
    # Abre la valvula en QW96 a un porcentaje determinado
    valvulap = max([0,valvulap])
    valvulap = min([valvulap,100])
    valvula = ((7800 / 71.5) * (valvulap - 7.4)) + 6200
    escr_sal_ent(96,valvula)
    

def BombaQW80(motobombaHz):
    #Poner bomba en 60Hz salida QW80
    motobombaHz = max([0,motobombaHz])
    motobombaHz = min([motobombaHz,60]) 
    motobomba = motobombaHz * (22118 / 60) + 5530
    escr_sal_ent(80,motobomba)

def leerNivelPLCIW100():
    nivelplc = leer_ent_ent(100)
    nivelcm = ((60 / 15105) * (nivelplc - 10125)) + 20
    # print(nivelplc, nivelcm)
    return nivelcm

def leerPresionPLCIW96():
    presionplc = leer_ent_ent(96)
    # 0.64 5679
    # 0.4  5623
    mp = (0.64-0.4)/(5679-5623)
    presion =  mp* presionplc - mp *5623+0.4
    # print(presionplc, presion)
    return presion


def leerFlujoPLCIW98():
    flujoplc = leer_ent_ent(98)
    # 4.67 8773 
    # 16.52  16957 
    mf = (4.67-16.52)/(8773-16957)
    flujo =  mf* flujoplc - mf *16957+16.52
    # print(flujoplc, flujo)
    return flujo

def leerNada():
    return np.random.rand(1)[0]

## 3. Inicie el experimento.

El módulo multivariable tiene:
- 2 actuadores:
    - Motobomba
    - Válvula
- 3 sensores:
    - Flujo
    - Presión
    - Nivel

3.1 Escoja una variable como salida y un actuador para su experimento.

3.2 Ponga un punto de operación para el otro actuador.

**Establecer un punto para la bomba**
~~~
BombaQW80(60)
~~~

**Establecer un punto para la válvula**
~~~
AbrirValvulaQW96(100)
~~~

In [74]:
escr_sal_bool(0,5,1) #Habilitar RUN motobomba
# Complete para establecer un punto de operación
BombaQW80(20)
AbrirValvulaQW96(100)

ERROR:snap7.common:b' ISO : An error occurred during send TCP : Connection reset by peer'


Snap7Exception: b' ISO : An error occurred during send TCP : Connection reset by peer'

## 5. Defina la entrada del sistema

Esta vez se configura una señal de referencia $r(t)$ en forma de un escalón.

\begin{equation}
r(t) = \begin{cases} V_0 & \forall t<t_0 \\  V_f &  \forall t > t_0 \end{cases}
\end{equation}

Defina:
- Duración para el experimento $t_f$
- Valor inicial para el actuador $V_0$
- Valor final para el actuador $V_f$

Recuerde el efecto de discretizar las señales con un tiempo de muestreo $t_m$.

**Señal de referencia $r(t)$**

In [52]:
tm = 0.3 # periodo de muestreo
tf = 200 # tiempo final del experimento
t = np.linspace(0.0 ,tf, round(tf/tm) + 1)
t0,v0,vf = 30,20,60
r = np.where(t>=t0,vf,v0)
plt.plot(t,r)


## 6. Realimente el sistema e implemente el PID



- Use los valores definidos para $r(t)$ y las lecturas $y(t)$ del sensor para calcular el error $e(t)$ como:

$$e(t)=r(t)-y(t)$$

- Envíe hacia el actuador una señal $u(t)$ generada a partir de la señal $e(t)$ como:
$$u(t) = k_c \left ( e(t) + \frac{1}{\tau_i} \int_0 ^t e(\tau)d\tau + \tau_d \frac{de}{dt}\right )$$

- Guarde la información necesaria. Para esto puede crear un archivo `csv` a partir de un **dataframe**.

Para leer los sensores se han definido las siguientes funciones:

~~~
leerNivelPLCIW100()
leerPresionPLCIW96()
leerFlujoPLCIW98()
~~~

In [1]:
# Define los parámetros del controlador
kc,Ti,Td = 1,1,1
## Inicia listas para salida y(t) , error e(t) , señal de control u(t)
y,e,u = ([],[],[])
e3,e2,e1,Inte = (0,0,0,0)
print('Para abortar el experimento presione ESPACIO')
for idt,ti in enumerate(t):
    ref_t = r[idt] # Referencia en instante  actual
    y_t = leerPresionPLCIW96() # Salida del sistema en instante actual
    e_t = ref_t - y_t # Error  en instante actual
    Inte = e_t*tm + Inte
    dedt = (e_t - e1)/tm
    ## Estrategia de control
    # modifique la estrategia de control
    u_t = kc*(e_t + (1/Ti)*Inte + Td*dedt) 
    # Señal de control en instante actual
    ##
    # actualiza datos de error
    e3,e2,e1 = (e2,e1,e_t)
    ## enviar señal de control al actuador
    #BombaQW80(u_t) 
    AbrirValvulaQW96(u_t)
    ## Agrupar datos para guardar
    y.append(y_t)
    e.append(e_t)
    u.append(u_t)
    ## Aborta
    if keyboard.is_pressed(' '):
        print('Abortando a los ' + str(ti) +' segundos')
        escr_sal_bool(0,5,0)
        AbrirValvulaQW96(0)
        y = y + [np.nan]*(len(r)-len(y))
        e = e + [np.nan]*(len(r)-len(e))
        u = u + [np.nan]*(len(r)-len(u))
        break
    time.sleep(tm)
#Termina experimento
escr_sal_bool(0,5,0)
AbrirValvulaQW96(0)
# Gráfica de entrada y salida
plt.plot(t,r,t,y)

In [69]:
# Guarda datos
df = pd.DataFrame(data={'Tiempo': t, 'Referencia': r, 
                        'Error': e, 'Señal de control': u,
                        'Salida': y })
archivo = "Datos/Nombre" + ".csv"
df.to_csv(archivo, index=False)

In [1]:
## Escriba su código

## Conclusiones
- Aquí
- Otra